In [62]:
from one.api import ONE
one = ONE()
assert not one.offline, 'ONE must be connect to Alyx for searching imaging sessions'
from datetime import datetime

import numpy as np
query = 'field_of_view__imaging_type__name,mesoscope'
eids = one.search(procedures='Imaging', django=query, query_type='remote')

len(eids)

319

In [83]:


det = one.get_details(eids[0], full=True)
print(eids[0])
# det
# det.keys()
session_start = det.get('start_time')
session_end = det.get('end_time')
print(f"{session_start}, {session_end}")
start_dt = datetime.fromisoformat(session_start.replace('Z', '+00:00'))
end_dt = datetime.fromisoformat(session_end.replace('Z', '+00:00'))

duration_seconds = (end_dt - start_dt).total_seconds()
duration_hours = duration_seconds / 3600

if duration_hours < 1:
    print(f"Session duration: {duration_seconds:.0f} seconds")
print(f"Session duration: {duration_hours:.2f} hours")  
# det['notes']

61f260e7-b5d3-4865-a577-bcfc53fda8a8
2025-04-09T12:10:56.318000, 2025-04-09T13:16:00.121000
Session duration: 1.08 hours


In [91]:

query = 'field_of_view__imaging_type__name,mesoscope'
eids = one.search(procedures='Imaging', django=query, query_type='remote')

print(f"Number of mesoscope imaging sessions: {len(eids)}")

# Let's create a list to store session information
good_sessions = []

# Define a limit to avoid processing all sessions
session_limit = 310  # Adjust as needed

for i, eid in enumerate(eids[:session_limit]):
    print(f"\nProcessing session {i+1}/{session_limit} (ID: {eid})")
    
    try:
        # Get session metadata
        session_info = one.get_details(eid, full=True)
        task_protocol = session_info.get('task_protocol', '')
        
        # Check if the session is biasedChoiceWorld or passiveChoiceWorld
        if not ('biasedChoiceWorld' in task_protocol or 'passiveChoiceWorld' in task_protocol):
            print("- Not biasedChoiceWorld or passiveChoiceWorld, skipping")
            continue
            
        # Get session duration
        # print(session_info)
        session_start = session_info['start_time']
        session_end = session_info['end_time']
        duration_hours = None
        if session_start and session_end:
            start_dt = datetime.fromisoformat(session_start.replace('Z', '+00:00'))
            end_dt = datetime.fromisoformat(session_end.replace('Z', '+00:00'))
            # Calculate duration
            duration_seconds = (end_dt - start_dt).total_seconds()
            duration_hours = duration_seconds / 3600
            
            if duration_hours < 1:
                print(f"- Session too short ({duration_hours:.2f} hours), skipping")
                continue
            else:
                print(f"Session duration: {duration_hours:.2f} hours")
        else:
            print("- Missing session start or end time, skipping")
            continue
    
        # List all datasets to identify FOVs
        all_datasets = one.list_datasets(eid)
        
        # Find all FOVs in this session
        fov_collections = set()
        for dataset in all_datasets:
            if 'mpci' in dataset and 'FOV_' in dataset:
                # Extract collection path (e.g., alf/FOV_00)
                parts = dataset.split('/')
                if len(parts) >= 2:  # Make sure we have at least collection/dataset
                    collection = '/'.join(parts[:-1])  # Everything except the last part
                    fov_collections.add(collection)
        
        # Sort FOV collections for consistent order
        fov_collections = sorted(list(fov_collections))
        print(f"Found {len(fov_collections)} FOV collections: {fov_collections}")
        
        # Process each FOV collection
        good_fovs = []
        
        for collection in fov_collections:
            try:
                print(f"Checking FOV collection: {collection}")
                
                # List all datasets in this FOV
                fov_datasets = one.list_datasets(eid, collection=collection)
                
                # Check if necessary datasets exist
                has_roi_activity = any('mpci.ROIActivityDeconvolved' in ds for ds in fov_datasets)
                has_times = any('mpci.times' in ds for ds in fov_datasets)
                has_bad_frames = any('mpci.badFrames' in ds for ds in fov_datasets)
                has_frame_qc = any('mpci.mpciFrameQC' in ds for ds in fov_datasets)
                
                # Print which datasets are present
                print(f"Required datasets present: ROIActivity={has_roi_activity}, times={has_times}, badFrames={has_bad_frames}, frameQC={has_frame_qc}")
                
                if not (has_roi_activity and has_times and has_bad_frames and has_frame_qc):
                    print(f"- Missing required datasets, skipping")
                    continue
                
                # Only load QC data to check for issues (much smaller files)
                bad_frames = one.load_dataset(eid, 'mpci.badFrames', collection=collection)
                frame_qc = one.load_dataset(eid, 'mpci.mpciFrameQC', collection=collection)
                
                if bad_frames is None or frame_qc is None:
                    print(f"- Could not load QC datasets, skipping")
                    continue
                
                # Check if there are any bad frames
                bad_frame_count = np.sum(bad_frames)
                qc_issue_count = np.sum(frame_qc)
                
                # Allow up to 5% bad frames
                total_frames = len(bad_frames)
                bad_frame_percentage = (bad_frame_count / total_frames) * 100 if total_frames > 0 else 0
                qc_issue_percentage = (qc_issue_count / total_frames) * 100 if total_frames > 0 else 0
                
                max_allowed_percentage = 5
                
                if bad_frame_percentage > max_allowed_percentage or qc_issue_percentage > max_allowed_percentage:
                    print(f"- {collection} has too many QC issues: {bad_frame_percentage:.1f}% bad frames, {qc_issue_percentage:.1f}% QC issues")
                    continue
                
                # For ROI count, we'll use a less data-intensive approach
                # Just load the ROI Activity shape to get the count
                try:
                    # This will only load the array metadata, not the full array
                    roi_activity = one.load_dataset(eid, 'mpci.ROIActivityDeconvolved', collection=collection)
                    roi_count = roi_activity.shape[1]  # Number of columns = number of ROIs
                    print(f"Found {roi_count} ROIs in activity matrix")
                    
                    # If there are too few ROIs, skip this FOV
                    if roi_count < 50:  # Adjust this threshold as needed
                        print(f"- Too few ROIs ({roi_count}), skipping")
                        continue
                    
                except Exception as e:
                    print(f"- Error getting ROI count: {str(e)}")
                    continue
                
                # This FOV passed all checks
                good_fovs.append({
                    'collection': collection,
                    'roi_count': roi_count,
                    'frame_count': total_frames,
                    'bad_frame_percentage': bad_frame_percentage,
                    'qc_issue_percentage': qc_issue_percentage
                })
                
                print(f"+ Good FOV! {collection}: {roi_count} ROIs, {total_frames} frames")
                
            except Exception as e:
                print(f"- Error processing FOV {collection}: {str(e)}")
        
        # Only add session if it has at least one good FOV
        if good_fovs:
            subject = session_info.get('subject', 'unknown')
            date = session_info.get('start_time', 'unknown')
            
            # Add session to good sessions list
            good_sessions.append({
                'eid': eid,
                'subject': subject,
                'date': date,
                'duration_hours': duration_hours if duration_hours else 0,
                'task_protocol': task_protocol,
                'fov_count': len(good_fovs),
                'total_rois': sum(fov['roi_count'] for fov in good_fovs),
                'fovs': good_fovs
            })
            
            print(f"+ Good session! {subject}, {duration_hours:.2f} hours, {len(good_fovs)} good FOVs, {sum(fov['roi_count'] for fov in good_fovs)} total ROIs")
        else:
            print("- No good FOVs found in this session")
            
    except Exception as e:
        print(f"- Error processing session: {str(e)}")
    
    if len(good_sessions) >= 10:  # Stop once we have found enough good sessions
        print("Found 10 good sessions, stopping search")
        break

# Create a DataFrame of good sessions
if good_sessions:
    # Create main sessions dataframe
    sessions_df = pd.DataFrame([{
        'eid': s['eid'],
        'subject': s['subject'],
        'date': s['date'],
        'duration_hours': s['duration_hours'],
        'task_protocol': s['task_protocol'],
        'good_fov_count': s['fov_count'],
        'total_rois': s['total_rois']
    } for s in good_sessions])
    
    print("\nGood sessions found:")
    print(sessions_df)
    
    # Create FOVs dataframe for the first session
    if good_sessions and 'fovs' in good_sessions[0]:
        first_session_fovs = pd.DataFrame(good_sessions[0]['fovs'])
        print(f"\nFOVs in first good session ({good_sessions[0]['eid']}):")
        print(first_session_fovs)
    
    print("\nExample code to load and process data from all FOVs in the first good session:")
    if good_sessions:
        session = good_sessions[0]
        eid = session['eid']
        fov_collections = [fov['collection'] for fov in session['fovs']]
        
        
else:
    print("No good sessions found. Consider relaxing the criteria.")

Number of mesoscope imaging sessions: 319


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [97]:
df= pd.read_csv('good_mesoscope_sessions.csv')
df['eid'].values

array(['61f260e7-b5d3-4865-a577-bcfc53fda8a8',
       '5ea6bb9b-6163-4e8a-816b-efe7002666b0',
       'd682fbb9-4fd9-450c-9812-e7c32f754a42',
       'dd45de54-6b3c-443a-949d-e74954fc33b2',
       '3f8c0614-5bd4-4c35-b13e-042e25823a64',
       '7ba8c055-a1a8-4cf9-9acf-1c43f2a99dbf',
       '00fbcc3f-9ee1-4993-9f94-ce8be232752c',
       '727b4408-49d4-4742-85c8-323cd187f5fa',
       'b7f5b6ee-5ec9-4c8f-8ae7-d07f6303f223',
       'b52efb6c-f3b8-41c7-895f-365be2b3fc78'], dtype=object)

In [ ]:
from one.api import ONE
import numpy as np
import pandas as pd
from datetime import datetime
import time
import gc  # Garbage collection for memory management

# Connect to the ONE API
one = ONE()
assert not one.offline, 'ONE must be connect to Alyx for searching imaging sessions'
# Search for mesoscope sessions using the correct query
query = 'field_of_view__imaging_type__name,mesoscope'
eids = one.search(procedures='Imaging', django=query, query_type='remote')

print(f"Number of mesoscope imaging sessions: {len(eids)}")

# Let's create a list to store session information
good_sessions = []

# Define a limit to avoid processing all sessions
session_limit = len(eids)  # Process all sessions
batch_size = 50  # Process this many at a time
pause_seconds = 5  # Pause between batches to avoid rate limiting

# Process sessions in batches
for batch_start in range(0, min(session_limit, len(eids)), batch_size):
    batch_end = min(batch_start + batch_size, session_limit)
    print(f"\nProcessing batch {batch_start//batch_size + 1}, sessions {batch_start+1}-{batch_end}")
    
    for i in range(batch_start, batch_end):
        eid = eids[i]
        print(f"\nProcessing session {i+1}/{session_limit} (ID: {eid})")
        
        try:
            # Get session metadata with full details
            session_info = one.get_details(eid, full=True)
            task_protocol = session_info.get('task_protocol', '')
            
            # Check if the session is biasedChoiceWorld or passiveChoiceWorld
            if not ('biasedChoiceWorld' in task_protocol or 'passiveChoiceWorld' in task_protocol):
                print("- Not biasedChoiceWorld or passiveChoiceWorld, skipping")
                continue
                
            # Get session duration
            session_start = session_info.get('start_time')
            session_end = session_info.get('end_time')
            
            duration_hours = None
            if session_start and session_end:
                try:
                    # Parse datetime strings
                    start_dt = datetime.fromisoformat(session_start.replace('Z', '+00:00'))
                    end_dt = datetime.fromisoformat(session_end.replace('Z', '+00:00'))
                    
                    # Calculate duration
                    duration_seconds = (end_dt - start_dt).total_seconds()
                    duration_hours = duration_seconds / 3600
                    
                    if duration_hours < 1:
                        print(f"- Session too short ({duration_hours:.2f} hours), skipping")
                        continue
                    else:
                        print(f"Session duration: {duration_hours:.2f} hours")
                except Exception as e:
                    print(f"- Error parsing session times: {str(e)}, skipping")
                    continue
            else:
                print("- Missing session start or end time, skipping")
                continue
        
            # List all datasets to identify FOVs
            all_datasets = one.list_datasets(eid)
            
            # Find all FOVs in this session
            fov_collections = set()
            for dataset in all_datasets:
                if 'mpci' in dataset and 'FOV_' in dataset:
                    # Extract collection path (e.g., alf/FOV_00)
                    parts = dataset.split('/')
                    if len(parts) >= 2:  # Make sure we have at least collection/dataset
                        collection = '/'.join(parts[:-1])  # Everything except the last part
                        fov_collections.add(collection)
            
            # Sort FOV collections for consistent order
            fov_collections = sorted(list(fov_collections))
            print(f"Found {len(fov_collections)} FOV collections: {fov_collections}")
            
            # Process each FOV collection
            good_fovs = []
            
            for collection in fov_collections:
                try:
                    print(f"Checking FOV collection: {collection}")
                    
                    # List all datasets in this FOV
                    fov_datasets = one.list_datasets(eid, collection=collection)
                    
                    # Check if necessary datasets exist
                    has_roi_activity = any('mpci.ROIActivityDeconvolved' in ds for ds in fov_datasets)
                    has_times = any('mpci.times' in ds for ds in fov_datasets)
                    has_bad_frames = any('mpci.badFrames' in ds for ds in fov_datasets)
                    has_frame_qc = any('mpci.mpciFrameQC' in ds for ds in fov_datasets)
                    
                    # Print which datasets are present
                    print(f"Required datasets present: ROIActivity={has_roi_activity}, times={has_times}, badFrames={has_bad_frames}, frameQC={has_frame_qc}")
                    
                    if not (has_roi_activity and has_times and has_bad_frames and has_frame_qc):
                        print(f"- Missing required datasets, skipping")
                        continue
                    
                    # Only load QC data to check for issues (much smaller files)
                    bad_frames = one.load_dataset(eid, 'mpci.badFrames', collection=collection)
                    frame_qc = one.load_dataset(eid, 'mpci.mpciFrameQC', collection=collection)
                    
                    if bad_frames is None or frame_qc is None:
                        print(f"- Could not load QC datasets, skipping")
                        continue
                    
                    # Check if there are any bad frames
                    bad_frame_count = np.sum(bad_frames)
                    qc_issue_count = np.sum(frame_qc)
                    
                    # Allow up to 5% bad frames
                    total_frames = len(bad_frames)
                    bad_frame_percentage = (bad_frame_count / total_frames) * 100 if total_frames > 0 else 0
                    qc_issue_percentage = (qc_issue_count / total_frames) * 100 if total_frames > 0 else 0
                    
                    max_allowed_percentage = 5
                    
                    if bad_frame_percentage > max_allowed_percentage or qc_issue_percentage > max_allowed_percentage:
                        print(f"- {collection} has too many QC issues: {bad_frame_percentage:.1f}% bad frames, {qc_issue_percentage:.1f}% QC issues")
                        continue
                    
                    # For ROI count, we'll use a less data-intensive approach
                    # Just load the ROI Activity shape to get the count
                    try:
                        # This will only load the array metadata, not the full array
                        roi_activity = one.load_dataset(eid, 'mpci.ROIActivityDeconvolved', collection=collection)
                        roi_count = roi_activity.shape[1]  # Number of columns = number of ROIs
                        print(f"Found {roi_count} ROIs in activity matrix")
                        
                        # If there are too few ROIs, skip this FOV
                        if roi_count < 50:  # Adjust this threshold as needed
                            print(f"- Too few ROIs ({roi_count}), skipping")
                            continue
                        
                    except Exception as e:
                        print(f"- Error getting ROI count: {str(e)}")
                        continue
                    
                    # This FOV passed all checks
                    good_fovs.append({
                        'collection': collection,
                        'roi_count': roi_count,
                        'frame_count': total_frames,
                        'bad_frame_percentage': bad_frame_percentage,
                        'qc_issue_percentage': qc_issue_percentage
                    })
                    
                    print(f"+ Good FOV! {collection}: {roi_count} ROIs, {total_frames} frames")
                    
                    # Clean up to free memory
                    del bad_frames, frame_qc, roi_activity
                    gc.collect()
                    
                except Exception as e:
                    print(f"- Error processing FOV {collection}: {str(e)}")
            
            # Only add session if it has at least one good FOV
            if good_fovs:
                subject = session_info.get('subject', 'unknown')
                date = session_info.get('start_time', 'unknown')
                
                # Add session to good sessions list
                good_sessions.append({
                    'eid': eid,
                    'subject': subject,
                    'date': date,
                    'duration_hours': duration_hours if duration_hours else 0,
                    'task_protocol': task_protocol,
                    'fov_count': len(good_fovs),
                    'total_rois': sum(fov['roi_count'] for fov in good_fovs),
                    'fovs': good_fovs
                })
                
                print(f"+ Good session! {subject}, {duration_hours:.2f} hours, {len(good_fovs)} good FOVs, {sum(fov['roi_count'] for fov in good_fovs)} total ROIs")
            else:
                print("- No good FOVs found in this session")
                
        except Exception as e:
            print(f"- Error processing session: {str(e)}")
        
        # Clean up to free memory
        gc.collect()
        
        if len(good_sessions) >= 10:  # Stop once we have found enough good sessions
            print("Found 10 good sessions, stopping search")
            break
    
    # Save progress after each batch
    if good_sessions:
        print(f"\nSaving progress after batch: {len(good_sessions)} good sessions found so far")
        # Save to CSV
        sessions_df = pd.DataFrame([{
            'eid': s['eid'],
            'subject': s['subject'],
            'date': s['date'],
            'duration_hours': s['duration_hours'],
            'task_protocol': s['task_protocol'],
            'good_fov_count': s['fov_count'],
            'total_rois': s['total_rois']
        } for s in good_sessions])
        
        sessions_df.to_csv('good_mesoscope_sessions.csv', index=False)
    
    # Pause between batches to avoid rate limiting
    if batch_end < session_limit and batch_end < len(eids):
        print(f"Pausing for {pause_seconds} seconds before next batch...")
        time.sleep(pause_seconds)
    
    # Break the outer loop too if we've found enough sessions
    if len(good_sessions) >= 10:
        break

# Create a DataFrame of good sessions
if good_sessions:
    # Create main sessions dataframe
    sessions_df = pd.DataFrame([{
        'eid': s['eid'],
        'subject': s['subject'],
        'date': s['date'],
        'duration_hours': s['duration_hours'],
        'task_protocol': s['task_protocol'],
        'good_fov_count': s['fov_count'],
        'total_rois': s['total_rois']
    } for s in good_sessions])
    
    print("\nGood sessions found:")
    print(sessions_df)
    
    # Create FOVs dataframe for the first session
    if good_sessions and 'fovs' in good_sessions[0]:
        first_session_fovs = pd.DataFrame(good_sessions[0]['fovs'])
        print(f"\nFOVs in first good session ({good_sessions[0]['eid']}):")
        print(first_session_fovs)
    
    # Save the final results
    sessions_df.to_csv('good_mesoscope_sessions_final.csv', index=False)
   
else:
    print("No good sessions found. Consider relaxing the criteria.")

Number of mesoscope imaging sessions: 319

Processing batch 1, sessions 1-50

Processing session 1/319 (ID: 61f260e7-b5d3-4865-a577-bcfc53fda8a8)
Session duration: 1.08 hours
Found 8 FOV collections: ['alf/FOV_00', 'alf/FOV_01', 'alf/FOV_02', 'alf/FOV_03', 'alf/FOV_04', 'alf/FOV_05', 'alf/FOV_06', 'alf/FOV_07']
Checking FOV collection: alf/FOV_00
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True
Found 3033 ROIs in activity matrix
+ Good FOV! alf/FOV_00: 3033 ROIs, 19081 frames
Checking FOV collection: alf/FOV_01
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True
Found 3470 ROIs in activity matrix
+ Good FOV! alf/FOV_01: 3470 ROIs, 19081 frames
Checking FOV collection: alf/FOV_02
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True
Found 4491 ROIs in activity matrix
+ Good FOV! alf/FOV_02: 4491 ROIs, 19081 frames
Checking FOV collection: alf/FOV_03
Required datasets present: ROIActivi

(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_00\mpci.badFrames.npy: 100%|██████████| 19.0k/19.0k [00:00<00:00, 84.8kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_00\mpci.mpciFrameQC.npy: 100%|██████████| 75.5k/75.5k [00:00<00:00, 193kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_00\mpci.ROIActivityDeconvolved.npy: 100%|██████████| 266M/266M [00:06<00:00, 40.2MB/s] 


Found 3528 ROIs in activity matrix
+ Good FOV! alf/FOV_00: 3528 ROIs, 18842 frames
Checking FOV collection: alf/FOV_01
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True


(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_01\mpci.badFrames.npy: 100%|██████████| 19.0k/19.0k [00:00<00:00, 88.7kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_01\mpci.mpciFrameQC.npy: 100%|██████████| 75.5k/75.5k [00:00<00:00, 165kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-12-18\001\alf\FOV_01\mpci.ROIActivityDeconvolved.npy: 100%|██████████| 201M/201M [00:05<00:00, 38.7MB/s] 


Found 2672 ROIs in activity matrix
+ Good FOV! alf/FOV_01: 2672 ROIs, 18842 frames
Checking FOV collection: alf/FOV_02
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True


...

Found 2977 ROIs in activity matrix
+ Good FOV! alf/FOV_07: 2977 ROIs, 18672 frames
+ Good session! SP061, 1.05 hours, 8 good FOVs, 24898 total ROIs

Processing session 60/319 (ID: 43ed4326-9b2e-4260-ba24-2757edb459a9)
- Not biasedChoiceWorld or passiveChoiceWorld, skipping

Processing session 61/319 (ID: 1c682c4a-d7a5-4792-89f9-53d7e9d3cf89)
- Not biasedChoiceWorld or passiveChoiceWorld, skipping

Processing session 62/319 (ID: 00fbcc3f-9ee1-4993-9f94-ce8be232752c)
Session duration: 1.20 hours
Found 8 FOV collections: ['alf/FOV_00', 'alf/FOV_01', 'alf/FOV_02', 'alf/FOV_03', 'alf/FOV_04', 'alf/FOV_05', 'alf/FOV_06', 'alf/FOV_07']
Checking FOV collection: alf/FOV_00
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True


(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_00\mpci.badFrames.npy: 100%|██████████| 20.3k/20.3k [00:00<00:00, 71.3kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_00\mpci.mpciFrameQC.npy: 100%|██████████| 80.8k/80.8k [00:00<00:00, 188kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_00\mpci.ROIActivityDeconvolved.npy: 100%|██████████| 391M/391M [00:11<00:00, 32.9MB/s] 


Found 4848 ROIs in activity matrix
+ Good FOV! alf/FOV_00: 4848 ROIs, 20164 frames
Checking FOV collection: alf/FOV_01
Required datasets present: ROIActivity=True, times=True, badFrames=True, frameQC=True


(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_01\mpci.badFrames.npy: 100%|██████████| 20.3k/20.3k [00:00<00:00, 94.1kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_01\mpci.mpciFrameQC.npy: 100%|██████████| 80.8k/80.8k [00:00<00:00, 172kB/s]
(S3) C:\Users\Gugu\Downloads\ONE\alyx.internationalbrainlab.org\cortexlab\Subjects\SP061\2024-11-08\001\alf\FOV_01\mpci.ROIActivityDeconvolved.npy: 100%|██████████| 181M/181M [00:06<00:00, 27.9MB/s] 


KeyboardInterrupt: 

In [3]:
FOVs = one.alyx.rest('fields-of-view', 'list', imaging_type='mesoscope' ) #, atlas_acronym='*')
FOVs

In [4]:
eids = one.search(procedures='Imaging', django=query, query_type='remote', atlas_acronym='VISp1')
det = one.get_details(eids[0], full=True)
FOVs = det['field_of_view']
print(FOVs[0])

{'id': '44fc4626-23b7-4b9c-8d02-3183db9ff305', 'imaging_type': 'mesoscope', 'location': [{'id': '3fae99ae-4c9e-4336-b5d8-c8174e8a0394', 'brain_region': [558, 981, 873], 'coordinate_system': 'IBL-Allen', 'provenance': 'E', 'x': [4589.930370338333, 4672.17497755925, 4224.141273072956, 4285.824728488641], 'y': [-687.3665579357845, -1367.3115498075197, -595.7573752147621, -1278.8656212103792], 'z': [-2404.498088065678, -2216.6800025770453, -1778.5020061386715, -1612.2994904965633], 'n_xyz': [512, 512, 1], 'default_provenance': True, 'auto_datetime': '2025-04-10T03:53:27.544000'}], 'name': 'FOV_00', 'session': '61f260e7-b5d3-4865-a577-bcfc53fda8a8', 'stack': None, 'datasets': []}


In [9]:
one = ONE()
eid = '009aad6f-1881-46ca-b763-bc4ec8972692'

fov_folders = one.list_collections(eid, collection='alf/FOV_*')
fovs = sorted(map(lambda x: int(x[-2:]), fov_folders))
nFOV = len(fovs)

HTTPError: [Errno 404] /sessions/009aad6f-1881-46ca-b763-bc4ec8972692: 'No Session matches the given query.'

In [10]:
from one.api import ONE
one = ONE()
eid = '009aad6f-1881-46ca-b763-bc4ec8972692'

fov_folders = one.list_collections(eid, collection='alf/FOV_*')
fovs = sorted(map(lambda x: int(x[-2:]), fov_folders))
nFOV = len(fovs)

HTTPError: [Errno 404] /sessions/009aad6f-1881-46ca-b763-bc4ec8972692: 'No Session matches the given query.'